In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import math as mt
import tensorflow as tf

In [2]:
fp_padrao = 0.92
fp_padrao_Sup = 2-fp_padrao

In [3]:
def normaliza(s):
    if s >= 0:
       return s
    elif s < 0 :
       return s + 2

In [4]:
base = pd.read_excel('testeexport.xlsx')
base1 = base

base.drop(columns=['Freqüência', 'Ocorrência'],inplace=True)

base.rename(columns ={'Fat,Pot T': 'FP T', 
                            'Fat,Pot, R': 'FP R',
                            'Fat,Pot, S': 'FP S'}, inplace=True)
base = pd.pivot_table(base, index=['Data'])

In [5]:
base['Pot R KVA'] = (base['Corrente R'] * base['Tensão R'] * (mt.sqrt(3)))/1000
base['Pot S KVA'] = (base['Corrente S'] * base['Tensão S'] * (mt.sqrt(3)))/1000
base['Pot T KVA'] = (base['Corrente T'] * base['Tensão T'] * (mt.sqrt(3)))/1000

In [6]:
#Calculo KVAr usando numpy
base['Pot R KW'] = (base['Pot R KVA'] * (base['FP R']))
base['Pot S KW'] = (base['Pot S KVA'] * (base['FP S']))
base['Pot T KW'] = (base['Pot T KVA'] * (base['FP T']))

In [7]:
#Calculo KVAr usando numpy
base['Pot R KVAr'] = np.sqrt(np.power(base['Pot R KVA'],2) - np.power(base['Pot R KW'],2))
base['Pot S KVAr'] = np.sqrt(np.power(base['Pot S KVA'],2) - np.power(base['Pot S KW'],2))
base['Pot T KVAr'] = np.sqrt(np.power(base['Pot T KVA'],2) - np.power(base['Pot T KW'],2))

In [8]:
#Calculo Faltante Kvar
base['Pot R KVAr faltante'] = (base['Pot R KVAr']) - (np.sqrt(np.power(base['Pot R KVA'],2)-
                                                                      (np.power((base['Pot R KVA'] * (fp_padrao)),2))))
base['Pot S KVAr faltante'] = (base['Pot S KVAr']) - (np.sqrt(np.power(base['Pot S KVA'],2)-
                                                                      (np.power((base['Pot S KVA'] * (fp_padrao)),2))))
base['Pot T KVAr faltante'] = (base['Pot T KVAr']) - (np.sqrt(np.power(base['Pot T KVA'],2)-
                                                                      (np.power((base['Pot T KVA'] * (fp_padrao)),2))))

In [9]:
base["FP R"] = base["FP R"].apply(normaliza)
base["FP S"] = base["FP S"].apply(normaliza)
base["FP T"] = base["FP T"].apply(normaliza)

In [10]:
fpfases_abaixo = base.loc[((base["FP R"]< fp_padrao) | (base["FP R"]> fp_padrao_Sup))
                         | ((base["FP S"]< fp_padrao) | (base["FP S"]> fp_padrao_Sup))
                         | ((base["FP T"]< fp_padrao) | (base["FP T"]> fp_padrao_Sup))]

fpfases = base.loc[((base["FP R"]< fp_padrao) | (base["FP R"]> fp_padrao_Sup))
                         | ((base["FP S"]< fp_padrao) | (base["FP S"]> fp_padrao_Sup))
                         | ((base["FP T"]< fp_padrao) | (base["FP T"]> fp_padrao_Sup))]

In [11]:
maximoR = fpfases['Pot R KVAr faltante'].max()
maximoS = fpfases['Pot S KVAr faltante'].max()
maximoT = fpfases['Pot T KVAr faltante'].max()

In [12]:
base.head()

,Corrente R,Corrente S,Corrente T,FP R,FP S,FP T,Tensão R,Tensão S,Tensão T,Pot R KVA,...,Pot T KVA,Pot R KW,Pot S KW,Pot T KW,Pot R KVAr,Pot S KVAr,Pot T KVAr,Pot R KVAr faltante,Pot S KVAr faltante,Pot T KVAr faltante
Data,,,,,,,,,,,,,,,,,,,,,
2018-11-20 11:49:29,100.27,97.82,98.22,0.75,0.89,0.75,218.62,218.39,219.14,37.968333,...,37.280542,28.476250,32.931464,27.960406,25.113692,16.871290,24.658761,10.233205,2.369665,10.047832
2018-11-20 11:50:10,328.75,291.25,329.75,0.78,0.90,0.79,218.34,217.45,218.23,124.325351,...,124.640701,96.973774,98.725305,98.466154,77.800258,47.814847,76.418067,29.074870,4.823448,27.569088
2018-11-20 11:50:20,280.25,245.75,282.25,0.78,0.90,0.79,218.31,217.39,218.20,105.969254,...,106.671726,82.656018,83.279139,84.270664,66.313388,40.333928,65.401165,24.782092,4.068791,23.594557
2018-11-20 11:50:30,284.75,249.50,286.50,0.78,0.90,0.79,218.20,217.25,218.07,107.616560,...,108.213434,83.940917,84.495479,85.488613,67.344239,40.923028,66.346397,25.167333,4.128218,23.935565
2018-11-20 11:50:40,286.50,251.00,288.00,0.79,0.90,0.80,218.12,217.23,218.00,108.238245,...,108.745078,85.508214,84.995642,86.996062,66.361609,41.165268,65.247047,23.941053,4.152654,22.627854


In [17]:
def fases():
    exceldados = pd.ExcelWriter('FP Fases.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    base.round(5).to_excel(exceldados, sheet_name='GALLAS') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

def graficos():
    exceldados = pd.ExcelWriter('FP Fases gráficos.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    base.round(5).to_excel(exceldados, sheet_name='GALLAS') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

In [18]:
graficos()

In [19]:
fases()